In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
import os
import zipfile

files = os.listdir("zips")
for file in files:
    if file.split('.')[1] == 'zip':
        with zipfile.ZipFile(f'zips/{file}', 'r') as zip_ref:
            zip_ref.extractall(f'csvs/{file.split()[0]}')

In [18]:
os.listdir('csvs')

['developer_survey_2017.zip',
 '2015',
 '.DS_Store',
 '2011',
 '2016',
 'developer_survey_2020.zip',
 'developer_survey_2019.zip',
 'developer_survey_2018.zip']

### 2011 dataset

In [3]:
ROOT_CSV_PATH = 'csvs'
path = ROOT_CSV_PATH + '/' + '2011/2011 Stack Overflow Survey Results.csv'
path_2015 = f"{ROOT_CSV_PATH}/2015/2015 Stack Overflow Developer Survey Responses.csv"
path_2016 = f"{ROOT_CSV_PATH}/2016/2016 Stack Overflow Survey Responses.csv"
path_2017 = f"{ROOT_CSV_PATH}/2017/survey_results_public.csv"
path_2018 = f"{ROOT_CSV_PATH}/2018/survey_results_public.csv"
path_2019 = f"{ROOT_CSV_PATH}/2019/survey_results_public.csv"
path_2020 = f"{ROOT_CSV_PATH}/2020/survey_results_public.csv"


df_2011 = pd.read_csv(path)
df_2015 = pd.read_csv(path_2015)
df_2016 = pd.read_csv(path_2016)
df_2017 = pd.read_csv(path_2017)
df_2018 = pd.read_csv(path_2018)
df_2019 = pd.read_csv(path_2019)
df_2020 = pd.read_csv(path_2020)



/Users/admin/Projects/Msc/Data Mining/mining-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,108,121,196,197,198) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/admin/Projects/Msc/Data Mining/mining-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Objectives

+ Current Tech for each year (techDo)
+ prefered IDE/TextEditor
+ salary
+ OS
+ Gender
+ Age
+ Country

In [174]:
def append_to_df(mdf, series_list, years, value_label):
    
    for idx, series in enumerate(series_list):
        countries = series.value_counts().index.tolist()
        counts = series.value_counts().values.tolist()

        _df = pd.DataFrame(zip(countries, counts, [years[idx] for _ in range(len(counts))]),  
                                       columns=[value_label, 'count', 'year'])
        mdf = mdf.append(_df)
    return mdf

##### Countries

In [226]:
df_country = pd.DataFrame(columns=['country','count', 'year'])


all_countries_series = [df_2015.Country, df_2016.country, df_2017.Country, 
                        df_2018.Country, df_2019.Country, df_2020.Country]
all_year = [D'2015', '2016', '2017', '2018', '2019', '2020']

df_country = append_to_df(df_country ,all_countries_series ,all_year, 'country')

In [348]:
# Save to disk
df_country.to_csv("so_yearly/so_countries_2015-20.csv",index=False)

##### Gender

In [229]:
df_gender = pd.DataFrame(columns=['gender', 'count', 'year'])

all_age_series = [df_2015.Gender, df_2016.gender, df_2017.Gender, 
                df_2018.Gender, df_2019.Gender, df_2020.Gender]

df_gender = append_to_df(df_gender, all_age_series, all_year, 'gender')

In [347]:
df_gender.to_csv('so_yearly/so_gender_2015-20.csv', index=False)

##### Dev Type

In [18]:


def delimiter_sep_series(sep=','):
    
    def sep_series(series):
        series_map = {}
        for item in series.values:
            if type(item) == str:
                for i in item.split(sep):
                    key = i.strip()
                    series_map[key] = series_map.get(key, 0) + 1
                    
        return pd.Series(data=list(series_map.values()), index=list(series_map.keys()))
    
    return sep_series


df_devtypes = pd.DataFrame(columns=['dev_type','count', 'year'])
all_devtypes = [df_2015.Occupation.value_counts(), df_2016.occupation.value_counts(),
               df_2017.DeveloperType.agg(delimiter_sep_series(';')), df_2018.DevType.agg(delimiter_sep_series(';')),
               df_2019.DevType.agg(delimiter_sep_series(';')), df_2020.DevType.agg(delimiter_sep_series(';'))]

for idx, series in enumerate(all_devtypes):
    dev = series.index.tolist()
    counts = series.values.tolist()

    _df = pd.DataFrame(zip(dev, counts, [all_year[idx] for _ in range(len(counts))]),  
                                   columns=['dev_type', 'count', 'year'])
    df_devtypes = df_devtypes.append(_df)

AttributeError: 'DataFrame' object has no attribute 'Occupation'

In [350]:
df_devtypes.to_csv('so_yearly/so_devtypes_2015-20.csv', index=False)

##### Languages

In [351]:
all_languages = [df_2016.tech_do.agg(delimiter_sep_series(';')),df_2017.HaveWorkedLanguage.agg(delimiter_sep_series(';')),
                 df_2018.LanguageWorkedWith.agg(delimiter_sep_series(';')),df_2019.LanguageWorkedWith.agg(delimiter_sep_series(';')),
                 df_2020.LanguageWorkedWith.agg(delimiter_sep_series(';'))]

df_languages = pd.DataFrame(columns=['languages','count', 'year'])

for idx, series in enumerate(all_languages):
    dev = series.index.tolist()
    counts = series.values.tolist()

    _df = pd.DataFrame(zip(dev, counts, [all_year[idx] for _ in range(len(counts))]), 
                       columns=['languages', 'count', 'year'])
    
    df_languages = df_languages.append(_df)

In [353]:
df_languages.to_csv('so_yearly/so_languages-2016-20.csv', index=False)

##### Database and Platform

In [356]:
def column_to_df(columns, col):
    all_cols = [
        df_2017[columns[0]].agg(delimiter_sep_series(';')),
        df_2018[columns[1]].agg(delimiter_sep_series(';')),
        df_2019[columns[2]].agg(delimiter_sep_series(';')),
        df_2020[columns[3]].agg(delimiter_sep_series(';'))
    ]

    df_cols = pd.DataFrame(columns=[col,'count', 'year'])

    for idx, series in enumerate(all_cols):
        dev = series.index.tolist()
        counts = series.values.tolist()
        all_year=['2017', '2018','2019','2020']
        _df = pd.DataFrame(zip(dev, counts, [all_year[idx] for _ in range(len(counts))]), 
                           columns=[col, 'count', 'year'])

        df_cols = df_cols.append(_df)

    return df_cols

df_databases = column_to_df(['HaveWorkedDatabase', 'DatabaseWorkedWith', 'DatabaseWorkedWith', 'DatabaseWorkedWith'], 'database')
df_platforms = column_to_df(['HaveWorkedPlatform', 'PlatformWorkedWith', 'PlatformWorkedWith', 'PlatformWorkedWith'], 'platform')


In [359]:
df_databases.to_csv('so_yearly/so_databases_2017-20.csv', index=False)
df_platforms.to_csv('so_yearly/so_platforms_2017-20.csv', index=False)

##### Frameworks

In [360]:
# Frameworks
columns = ['HaveWorkedFramework', 'FrameworkWorkedWith', 'WebFrameWorkedWith', 'MiscTechWorkedWith' , 'WebframeWorkedWith', 'MiscTechWorkedWith']
all_cols = [
    df_2017[columns[0]].agg(delimiter_sep_series(';')),
    df_2018[columns[1]].agg(delimiter_sep_series(';')),
    df_2019[columns[2]].agg(delimiter_sep_series(';')),
    df_2019[columns[3]].agg(delimiter_sep_series(';')),
    df_2020[columns[4]].agg(delimiter_sep_series(';')),
    df_2020[columns[5]].agg(delimiter_sep_series(';'))
]

df_frameworks = pd.DataFrame(columns=['frameworks','count', 'year'])

for idx, series in enumerate(all_cols):
    dev = series.index.tolist()
    counts = series.values.tolist()
    all_year=['2017', '2018','2019','2019','2020','2020']
    _df = pd.DataFrame(zip(dev, counts, [all_year[idx] for _ in range(len(counts))]), 
                       columns=['frameworks', 'count', 'year'])

    df_frameworks = df_frameworks.append(_df)

In [362]:
df_frameworks.to_csv('so_yearly/so_frameworks_2017-20.csv', index=False)

#### Plotly Animation

In [199]:
graph_df['count'].max()

20949

In [203]:
countries = df_country.query("year == '2020'").sort_values(by='count', ascending=False).country[:10].values.tolist()

graph_df = df_country[df_country.country.isin(countries)]
px.line(graph_df,
           animation_frame="year",
           x="country",
           y="count",
           hover_name="country",
            range_y=[0,25000]
           )

#### 2015
+ Country
+ Age
+ Gender
+ Occupation : DevType
+ Desktop Operating System : OS
+ Compensation: midpoint

In [93]:
df_2015.columns[df_2015.columns.str.contains('OS') == 1]

Index(['Current Lang & Tech: iOS', 'Future Lang & Tech: iOS'], dtype='object')

In [59]:
df_2015.head(2)


,Country,Age,Gender,Tabs or Spaces,Years IT / Programming Experience,Occupation,Desktop Operating System,Desktop Operating System: write-in,Current Lang & Tech: Android,Current Lang & Tech: Arduino,Current Lang & Tech: AngularJS,Current Lang & Tech: C,Current Lang & Tech: C++,Current Lang & Tech: C++11,Current Lang & Tech: C#,Current Lang & Tech: Cassandra,Current Lang & Tech: CoffeeScript,Current Lang & Tech: Cordova,Current Lang & Tech: Clojure,Current Lang & Tech: Cloud,Current Lang & Tech: Dart,Current Lang & Tech: F#,Current Lang & Tech: Go,Current Lang & Tech: Hadoop,Current Lang & Tech: Haskell,Current Lang & Tech: iOS,Current Lang & Tech: Java,Current Lang & Tech: JavaScript,Current Lang & Tech: LAMP,Current Lang & Tech: Matlab,Current Lang & Tech: MongoDB,Current Lang & Tech: Node.js,Current Lang & Tech: Objective-C,Current Lang & Tech: Perl,Current Lang & Tech: PHP,Current Lang & Tech: Python,Current Lang & Tech: R,Current Lang & Tech: Redis,Current Lang & Tech: Ruby,Current Lang & Tech: Rust,Current Lang & Tech: Salesforce,Current Lang & Tech: Scala,Current Lang & Tech: Sharepoint,Current Lang & Tech: Spark,Current Lang & Tech: SQL,Current Lang & Tech: SQL Server,Current Lang & Tech: Swift,Current Lang & Tech: Visual Basic,Current Lang & Tech: Windows Phone,Current Lang & Tech: Wordpress,Current Lang & Tech: Write-In,Future Lang & Tech: Android,Future Lang & Tech: Arduino,Future Lang & Tech: AngularJS,Future Lang & Tech: C,Future Lang & Tech: C++,Future Lang & Tech: C++11,Future Lang & Tech: C#,Future Lang & Tech: Cassandra,Future Lang & Tech: CoffeeScript,Future Lang & Tech: Cordova,Future Lang & Tech: Clojure,Future Lang & Tech: Cloud,Future Lang & Tech: Dart,Future Lang & Tech: F#,Future Lang & Tech: Go,Future Lang & Tech: Hadoop,Future Lang & Tech: Haskell,Future Lang & Tech: iOS,Future Lang & Tech: Java,Future Lang & Tech: JavaScript,Future Lang & Tech: LAMP,Future Lang & Tech: Matlab,Future Lang & Tech: MongoDB,Future Lang & Tech: Node.js,Future Lang & Tech: Objective-C,Future Lang & Tech: Perl,Future Lang & Tech: PHP,Future Lang & Tech: Python,Future Lang & Tech: R,Future Lang & Tech: Redis,Future Lang & Tech: Ruby,Future Lang & Tech: Rust,Future Lang & Tech: Salesforce,Future Lang & Tech: Scala,Future Lang & Tech: Sharepoint,Future Lang & Tech: Spark,Future Lang & Tech: SQL,Future Lang & Tech: SQL Server,Future Lang & Tech: Swift,Future Lang & Tech: Visual Basic,Future Lang & Tech: Windows Phone,Future Lang & Tech: Wordpress,Future Lang & Tech: Write-In,Training & Education: No formal training,Training & Education: On the job,Training & Education: Boot camp or night school,Training & Education: Online Class,Training & Education: Mentorship,Training & Education: Industry certification,"Training & Education: Some college, but no CS degree",Training & Education: BS in CS,Training & Education: Masters in CS,Training & Education: PhD in CS,Training & Education: Other,Compensation,Compensation: midpoint,Employment Status,Industry,Job Satisfaction,Purchasing Power,Remote Status,Changed Jobs in last 12 Months,Open to new job opportunities,Most important aspect of new job opportunity: Salary,Most important aspect of new job opportunity: Equity,Most important aspect of new job opportunity: Important decisions,Most important aspect of new job opportunity: Health insurance,Most important aspect of new job opportunity: Industry,Most important aspect of new job opportunity: Tech stack,Most important aspect of new job opportunity: Company size,Most important aspect of new job opportunity: Company stage,Most important aspect of new job opportunity: Work - Life balance,Most important aspect of new job opportunity: Advancement,Most important aspect of new job opportunity: Job title,Most important aspect of new job opportunity: Office location,Most important aspect of new job opportunity: Quality of colleagues,Most important aspect of new job opportunity: Company culture,Most important aspect of new job opportunity: Company re

In [134]:
countries = df_2015.Country.value_counts().index.tolist()
counts = df_2015.Country.value_counts().values.tolist()
year = '2015'

df_country_2015 = pd.DataFrame(zip(countries, counts, ['2015' for _ in range(len(counts))]),  columns=['country', 'count', 'year'])
df_country = df_country.append(df_country_2015)

#### 2016

+ tech_do 
+ dev_environment
+ desktop_os
+ country
+ age_range
+ gender
+ occupation : DevType
+ salary_midpoint

In [58]:
pd.set_option('display.max_columns', None)
df_2016.head(2)

,Unnamed: 0,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,occupation_group,experience_range,experience_midpoint,salary_range,salary_midpoint,big_mac_index,tech_do,tech_want,aliens,programming_ability,employment_status,industry,company_size_range,team_size_range,women_on_team,remote,job_satisfaction,job_discovery,dev_environment,commit_frequency,hobby,dogs_vs_cats,desktop_os,unit_testing,rep_range,visit_frequency,why_learn_new_tech,education,open_to_new_job,new_job_value,job_search_annoyance,interview_likelihood,how_to_improve_interview_process,star_wars_vs_star_trek,agree_tech,agree_notice,agree_problemsolving,agree_diversity,agree_adblocker,agree_alcohol,agree_loveboss,agree_nightcode,agree_legacy,agree_mars,important_variety,important_control,important_sameend,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow
0,1888,Facebook,Afghanistan,Southern Asia,Central Asia,20-24,22.0,Male,Programmer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4637,Facebook,Afghanistan,Southern Asia,Central Asia,30-34,32.0,Male,Developer; Engineer; Programmer,Mobile developer - iOS,"Mobile Dev (Android, iOS, WP & Multi-Platform)",6 - 10 years,8.0,"$40,000 - $50,000",45000.0,NaN,iOS; Objective-C,Swift,Yes,7.0,Employed full-time,Consulting,100-499 employees,15-20 people,3,I rarely work remotely,I love my job,I knew I wanted to work here. I sought out the...,TextMate; Xcode,Multiple times a day,1-2 hours per week,Cats,Mac OS X,No,1,Multiple times a day,To keep my skills up to date,I'm self-taught; On-the-job training; B.S. in ...,I am not interested in new job opportunities,Salary; Ability to make or influence important...,Taking time off work to interview,10%,Show me more live code; Introduce me to the te...,Star Wars,Agree completely,Agree completely,Agree completely,Agree somewhat,Agree completely,Neutral,Agree somewhat,Neutral,Agree somewhat,Disagree somewhat,This is very important,This is very important,This is somewhat important,This is very important,This is very important,I don't care about this,This is somewhat important,This is somewhat important,I don't care about this,I don't care about this,Unrealistic expectations; Interacting with non...,To get help for my job; Because I can't do my ...


#### 2017

+ VerionControl 
+ Gender
+ Salary
+ IDE
+ HaveWorkedLanguages
+ HaveWorkedDatabase
+ HaveWorkedFramework
+ HaveWorkedPlatform - for platform
+ Country

In [25]:
df_2017['HaveWorkedLanguage'].agg(delimiter_sep_series(';'))

Swift              2368
JavaScript        22875
Python            11704
Ruby               3324
SQL               18754
Java              14524
PHP               10290
Matlab             1569
R                  1634
Rust                416
CoffeeScript       1192
Clojure             391
Elixir              380
Erlang              281
Haskell             649
C#                12476
Objective-C        2349
C                  6974
C++                8155
Assembly           1823
VB.NET             2273
Perl               1585
Scala              1309
F#                  457
TypeScript         3488
Lua                1039
VBA                1574
Groovy             1193
Go                 1557
Smalltalk           327
Visual Basic 6     1071
Common Lisp         273
Dart                145
Julia               138
Hack                107
dtype: int64

In [57]:
df_2017.head(3)

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,InvestTimeTools,WorkPayCare,KinshipDevelopers,ChallengeMyself,CompetePeers,ChangeWorld,JobSeekingStatus,HoursPerWeek,LastNewJob,AssessJobIndustry,AssessJobRole,AssessJobExp,AssessJobDept,AssessJobTech,AssessJobProjects,AssessJobCompensation,AssessJobOffice,AssessJobCommute,AssessJobRemote,AssessJobLeaders,AssessJobProfDevel,AssessJobDiversity,AssessJobProduct,AssessJobFinances,ImportantBenefits,ClickyKeys,JobProfile,ResumePrompted,LearnedHiring,ImportantHiringAlgorithms,ImportantHiringTechExp,ImportantHiringCommunication,ImportantHiringOpenSource,ImportantHiringPMExp,ImportantHiringCompanies,ImportantHiringTitles,ImportantHiringEducation,ImportantHiringRep,ImportantHiringGettingThingsDone,Currency,Overpaid,TabsSpaces,EducationImportant,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,CousinEducation,WorkStart,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform,IDE,AuditoryEnvironment,Methodology,VersionControl,CheckInCode,ShipIt,OtherPeoplesCode,ProjectManagement,EnjoyDebugging,InTheZone,DifficultCommunication,CollaborateRemote,MetricAssess,EquipmentSatisfiedMonitors,EquipmentSatisfiedCPU,EquipmentSatisfiedRAM,EquipmentSatisfiedStorage,EquipmentSatisfiedRW,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a soft ""g,"" like ""jiff""",Strongly agree,Strongly agree,Agree,Disagree,Strongly agree,Agree,Agree,Disagree,Somewhat agree,Disagree,Strongly agree,Strongly agree,Strongly disagree,Agree,Agree,Disagree,Agree,"I'm not actively looking, but I am open to new...",0.0,Not applicable/ never,Very important,Very important,Important,Very important,Very important,Very important,Important,Very important,Very important,Very important,Very important,Very important,Somewhat important,Not very important,Somewhat important,Stock options; Vacation/days off; Remote options,Yes,Other,NaN,NaN,Important,Important,Important,Somewhat important,Important,Not very important,Not very important,Not at all important,Somewhat important,Very important,NaN,NaN,Tabs,NaN,Online course; Open source contributions,NaN,NaN,NaN,6:00 AM,Swift,Swift,NaN,NaN,NaN,NaN,iOS,iOS,Atom; Xcode,Turn on some music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somewhat satisfied,Not very satisfied,Not at all satisfied,Very satisfied,Satisfied,Not very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have created a CV or Developer Story on Stac...,9.0,Desktop; iOS app,At leas

#### 2018

+ LanguageWorkedWith
+ DatabaseWorkedWith
+ PlatformWorkedWith
+ FrameworkWorkedWith
+ IDE
+ Operating System
+ VersionControl
+ Gender
+ Country

In [54]:
df_2018.head(1)

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,UpdateCV,Currency,Salary,SalaryType,ConvertedSalary,CurrencySymbol,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,IDE,OperatingSystem,NumberMonitors,Methodology,VersionControl,CheckInCode,AdBlocker,AdBlockerDisable,AdBlockerReasons,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,3-5 years,3-5 years,Extremely satisfied,Extremely satisfied,Working as a founder or co-founder of my own c...,"I’m not actively looking, but I am open to new...",Less than a year ago,10.0,7.0,8.0,1.0,2.0,5.0,3.0,4.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,4.0,2.0,5.0,5.0,6.0,7.0,2.0,1.0,4.0,3.0,My job status or other personal status changed,NaN,NaN,Monthly,NaN,KES,Slack,One to three months,"Taught yourself a new language, framework, or ...",The official documentation and/or standards fo...,NaN,To build my professional network,Strongly agree,Strongly agree,Neither Agree nor Disagree,JavaScript;Python;HTML;CSS,JavaScript;Python;HTML;CSS,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,Redis;SQL Server;MySQL;PostgreSQL;Amazon RDS/A...,AWS;Azure;Linux;Firebase,AWS;Azure;Linux;Firebase,Django;React,Django;React,Komodo;Vim;Visual Studio Code,Linux-based,1,Agile;Scrum,Git,Multiple times per day,Yes,No,NaN,Strongly agree,Strongly agree,Strongly agree,Saw an online advertisement and then researche...,1.0,5.0,4.0,7.0,2.0,6.0,3.0,Artificial intelligence surpassing human intel...,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, and publicly",Upper management at the company/organization,Yes,10 (Very Likely),Multiple times per day,Yes,I have never participated in Q&A on Stack Over...,"No, I knew that Stack Overflow had a jobs boar...",Yes,NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Between 5:00 - 6:00 AM,9 - 12 hours,1 - 2 hours,Never,Standing desk,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy


#### 2019

+ Country
+ DevType
+ LanguageWorkedWith
+ DatabaseWorkedWith
+ PlatformWorkedWith
+ WebFrameWorkedWith | MiscTechWorkedWith
+ DevEnviron - IDE/TextEditor
+ OpSys
+ Gender
+ Age

In [55]:
df_2019.head(1)

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,"Taught yourself a new language, framework, or ...",NaN,NaN,4,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTML/CSS;Java;JavaScript;Python,C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL,SQLite,MySQL,MacOS;Windows,Android;Arduino;Windows,Django;Flask,Flask;jQuery,Node.js,Node.js,IntelliJ;Notepad++;PyCharm,Windows,I do not use containers,NaN,NaN,Yes,"Fortunately, someone else has that title",Yes,Twitter,Online,Username,2017,A few times per month or weekly,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,31-60 minutes,No,NaN,"No, I didn't know that Stack Overflow had a jo...","No, and I don't know what those are",Neutral,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult


#### 2020

+ Age
+ Country
+ DatabaseWorkedWith
+ LanguageWorkedWith
+ PlatformWorkedWith
+ MiscTechWorkedWith | WebframeWorkedWith
+ OpSys
+ DevType
+ ConvertedComp : Salary in USD

In [83]:
df_2020.head(4)

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,EUR,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,.NET Core;Xamarin,.NET;.NET Core,Microsoft Teams;Microsoft Azure;Trello,Confluence;Jira;Slack;Microsoft Azure;Trello,No,Somewhat important,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,Often: 1-2 days per week or more,Start a free trial;Ask developers I know/work ...,Amused,Stack Overflow (public Q&A for anyone who codes),Visit Stack Overflow;Go for a walk or other ph...,Windows,2 to 9 employees,Android;iOS;Kubernetes;Microsoft Azure;Windows,Windows,NaN,Straight / Heterosexual,No,"No, not at all",NaN,Multiple times per day,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,GBP,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Very dissatisfied,I am not interested in new job opportunities,Python;Swift,JavaScript;Swift,React Native;TensorFlow;Unity 3D,React Native,Github;Slack,Confluence;Jira;Github;Gitlab;Slack,NaN,NaN,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,NaN,NaN,Amused,Stack Overflow (public Q&A for anyone who code...,Visit Stack Overflow;Go for a walk or other ph...,MacOS,"1,000 to 4,999 employees",iOS;Kubernetes;Linux;MacOS,iOS,I have little or no influence,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Once a decade,NaN,NaN,No,NaN,NaN,NaN,Stack Overflow (public Q&A for anyone who codes),NaN,Linux-based,NaN,NaN,NaN,NaN,NaN,Yes,"Yes, somewhat",A few times per month or weekly,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,ALL,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,White or of European descent,Man,Flex time or a flexible schedule;Office enviro...,Slightly dissatisfied,"I’m not actively looking, but I am open to new...",NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,Not at all important/not necessary,Curious about other opportunities;Wanting to w...,NaN,Once a year,Not sure,Yes,Yes,Occasionally: 1-2 days per quarter but less th...,NaN,NaN,Stack Overflow (public Q&A for anyone who code...,NaN,Lin

In [92]:
df_2020.CompTotal.isna().value_counts()

False    34826
True     29635
Name: CompTotal, dtype: int64

In [8]:
df_2019.WebFrameWorkedWith.unique()

array(['Django;Flask', 'Django', nan, ...,
       'Django;Drupal;Express;Ruby on Rails',
       'ASP.NET;Express;Spring;Vue.js',
       'Angular/Angular.js;Drupal;jQuery;Laravel;React.js;Ruby on Rails'],
      dtype=object)

Developer, desktop or enterprise applications;Developer, full-stack
Developer, full-stack;Developer, mobile
nan
Designer;Developer, front-end;Developer, mobile
Developer, back-end;Developer, front-end;Developer, QA or test;DevOps specialist
Developer, back-end;Developer, desktop or enterprise applications;Developer, game or graphics
Developer, full-stack
Database administrator;Developer, full-stack;Developer, mobile
Developer, back-end;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, front-end
Designer;Developer, front-end
Designer;Developer, back-end
Developer, desktop or enterprise applications;Developer, front-end
Database administrator;Developer, back-end;Developer, desktop or enterprise applications;Developer, full-stack;Developer, QA or test;DevOps specialist
Developer, back-end
Developer, desktop or enterprise applications
Developer, full-stack;Engineer, data
Developer, back-end;Developer, desktop or enterprise applications;Dev

Academic researcher;Data scientist or machine learning specialist;Developer, embedded applications or devices;Scientist
Data scientist or machine learning specialist;Designer;Developer, back-end;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, front-end;Developer, full-stack;Developer, game or graphics;Developer, mobile;Developer, QA or test
Data or business analyst;Data scientist or machine learning specialist;Developer, back-end;Developer, front-end;Developer, full-stack;Developer, mobile
Developer, back-end;Developer, full-stack;Developer, QA or test;DevOps specialist
Developer, back-end;Developer, front-end;Developer, full-stack;Developer, game or graphics;Engineer, data
Developer, back-end;Developer, front-end;Engineer, data
Data or business analyst;Designer;Developer, back-end;Developer, front-end;Developer, full-stack;Developer, mobile;Engineer, data
Designer;Developer, back-end;Developer, front-end;Developer, full-stack;Develo

Developer, back-end;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, mobile;Engineer, data
Data or business analyst;Data scientist or machine learning specialist;Marketing or sales professional;Scientist
Developer, back-end;Developer, desktop or enterprise applications;Developer, front-end;Developer, full-stack;DevOps specialist;Engineer, site reliability;Engineering manager
Data or business analyst;Developer, front-end;Developer, full-stack;Engineer, data
Database administrator;Designer;Developer, back-end;Developer, front-end;Developer, full-stack;DevOps specialist;Engineering manager;System administrator
Academic researcher;Data or business analyst;DevOps specialist
Data or business analyst;Database administrator;Developer, back-end;Developer, desktop or enterprise applications;Developer, front-end;Developer, full-stack;Developer, mobile;Marketing or sales professional;Product manager;System administrator
Data or business analyst;D

Database administrator;Developer, back-end;Developer, front-end;Developer, full-stack;DevOps specialist;Educator;Engineering manager
Data scientist or machine learning specialist;Developer, back-end;DevOps specialist;Scientist
Developer, back-end;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, front-end;Developer, full-stack;Developer, game or graphics;Developer, mobile;Engineering manager;System administrator
Designer;Developer, back-end;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, front-end;Developer, full-stack;Developer, game or graphics;Developer, mobile;DevOps specialist;Educator
Developer, back-end;Developer, desktop or enterprise applications;Engineering manager;Senior executive/VP
Data or business analyst;Developer, front-end;Product manager
Developer, mobile;Developer, QA or test;Product manager
Designer;Developer, back-end;Developer, front-end;Developer, full-stack;De

Database administrator;Developer, desktop or enterprise applications;Developer, front-end;Developer, QA or test;System administrator
Academic researcher;Data scientist or machine learning specialist;Designer;Developer, desktop or enterprise applications;Developer, full-stack;Marketing or sales professional;Product manager
Academic researcher;Developer, back-end;Developer, embedded applications or devices;Scientist
Data or business analyst;Database administrator;Developer, back-end;Developer, front-end;Developer, full-stack;Scientist;System administrator
Database administrator;Designer;Developer, back-end;Developer, desktop or enterprise applications;Developer, full-stack;DevOps specialist;Educator;Engineer, site reliability;System administrator
Academic researcher;Data or business analyst;Data scientist or machine learning specialist;Database administrator;Designer;Developer, back-end;Developer, desktop or enterprise applications;Developer, front-end;Educator;System administrator
Devel

Developer, back-end;Developer, desktop or enterprise applications;Developer, front-end;Developer, full-stack;Developer, game or graphics;DevOps specialist;Educator
Data or business analyst;Database administrator;Designer;Developer, front-end;Developer, full-stack;Marketing or sales professional
Data scientist or machine learning specialist;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, full-stack;Developer, game or graphics;Developer, mobile
Academic researcher;Data or business analyst;Data scientist or machine learning specialist;Developer, back-end;DevOps specialist;Engineer, data;Scientist
Developer, front-end;Developer, mobile;Engineering manager;Product manager
Developer, back-end;Developer, desktop or enterprise applications;Developer, QA or test;System administrator
Academic researcher;Database administrator;Developer, back-end;Developer, desktop or enterprise applications;Engineer, data
Database administrator;Developer, back